In [65]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[4] appName=DFSession>

In [66]:
spark

### 1. Create a new Spark Session

In [67]:
sc.stop()

In [68]:
spark.stop()

In [69]:
from pyspark import SparkConf,SparkContext

config = SparkConf().setMaster("local[4]").setAppName("DFSession")

sc = SparkContext(conf=config)

In [70]:
sc

<SparkContext master=local[4] appName=DFSession>

In [71]:
from pyspark.sql import SparkSession
# Here spark searches for the spark session and if found get it else creates one.
spark = SparkSession.builder.appName("SQLSession").getOrCreate()

In [72]:
spark

In [73]:
!hdfs dfs -mkdir /flights

mkdir: `/flights': File exists


In [74]:
!hdfs dfs -put /home/hadoop/Downloads/raw_flight_data.csv /flights

put: `/flights/raw_flight_data.csv': File exists


In [75]:
flights_df = spark.read.csv("hdfs://localhost:9000/flights/raw_flight_data.csv", header = True, inferSchema=True)

In [76]:
flights_df.show(10)  # Show Data in a Tabular Format.

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

### Select 
    * It is used to select specific column from the dataset.

In [77]:
flights_df.select(['DayOfMonth','DepDelay','Carrier','OriginAirportID']).show()

+----------+--------+-------+---------------+
|DayOfMonth|DepDelay|Carrier|OriginAirportID|
+----------+--------+-------+---------------+
|        19|      -3|     DL|          11433|
|        19|       0|     DL|          14869|
|        19|      -4|     DL|          14057|
|        19|      28|     DL|          15016|
|        19|      -6|     DL|          11193|
|        19|      -1|     DL|          10397|
|        19|       0|     DL|          15016|
|        19|      15|     DL|          10397|
|        19|      33|     DL|          10397|
|        19|     323|     DL|          11278|
|        19|      -7|     DL|          14107|
|        19|      22|     DL|          11433|
|        19|      40|     DL|          11298|
|        19|      -2|     DL|          11433|
|        19|      71|     DL|          10397|
|        19|      75|     DL|          12451|
|        19|      -1|     DL|          12953|
|        19|      -3|     DL|          11433|
|        19|      31|     DL|     

In [78]:
flights_df.select(['Carrier']).distinct().show()

+-------+
|Carrier|
+-------+
|     UA|
|     AA|
|     EV|
|     B6|
|     DL|
|     OO|
|     F9|
|     YV|
|     US|
|     MQ|
|     HA|
|     AS|
|     FL|
|     VX|
|     WN|
|     9E|
+-------+



In [79]:
flights_df.columns

['DayofMonth',
 'DayOfWeek',
 'Carrier',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay']

In [80]:
flights_df.count() # Total number of records/ count of dataframes.

2719418

In [81]:
flights_df.select(['Carrier']).distinct().count() # shows the total count of distinct carriers

16

#### printSchema() 
        * Method prints schema name and schema datatypes

In [82]:
flights_df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



#### Where()
        * It is like filter() method. Where boolean conditions are checked to perform the operation

In [83]:
flights_df.where(flights_df.DepDelay > 0).first()

Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=15016, DestAirportID=11433, DepDelay=28, ArrDelay=24)

In [84]:
flights_df.where(flights_df.DepDelay > 0).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          11298|        11433|      40|      20|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          13204|      

In [85]:
flights_df.where((flights_df.DepDelay > 0) & (flights_df.ArrDelay > 0)).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          11298|        11433|      40|      20|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          13204|      

#### Filter()

In [86]:
flights_df.filter(flights_df.DepDelay > 0).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          11298|        11433|      40|      20|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          13204|      

#### isin()
        * Filter values from DataFrame by matching patterns.

In [87]:
flights_df.Carrier.isin('DL','F9','UA','9E')

Column<b'(Carrier IN (DL, F9, UA, 9E))'>

In [88]:
flights_df.where(flights_df.Carrier.isin('DL','F9','UA','9E')).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

* Read airports.csv file as Spark DataFrame

In [89]:
airports_df = spark.read.csv('file:///home/hadoop/Downloads/airports.csv',header=True,inferSchema=True)

In [90]:
airports_df.show()

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
|     10926|    Cordova|   AK|Merle K Mudhole S...|
|     14709|  Deadhorse|   AK|   Deadhorse Airport|
|     11336| Dillingham|   AK|  Dillingham Airport|
|     11630|  Fairbanks|   AK|Fairbanks Interna...|
|     11997|   Gustavus|   AK|    Gustavus Airport|
|     12523|     Juneau|   AK|Juneau International|
|     12819|  Ketchikan|   AK|Ketchikan Interna...|
|     10245|King Salmon|   AK| King Salmon Airport|
|     10170|     Kodiak|   AK|      Kodiak Airport|
|     13970|   Kotzebue|   AK| Ralph Wien Memorial|
|     13873|       Nome|   AK|        Nome Airport|
|     14256|

### Join()
        * To join two or more than two dataframes using conditions.

In [91]:
flights_airport_df =flights_df.join(airports_df, on= flights_df.OriginAirportID == airports_df.airport_id)

In [92]:
flights_airport_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+----------+-----------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|             city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+-----------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|          Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|   Salt Lake City|   UT|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|         Portland|   OR|Portland Internat...|
|        19|        5|     DL|          15016|        11433|      28|      24|     15016|        St. Louis|   MO|Lambert-St. Louis...|
|        19|        5|     DL|          11193|        1

In [93]:
flights_airport_df = flights_df.join(airports_df.select(['airport_id','name']),flights_df.OriginAirportID == airports_df.airport_id)

In [94]:
flights_airport_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|Portland Internat...|
|        19|        5|     DL|          15016|        11433|      28|      24|     15016|Lambert-St. Louis...|
|        19|        5|     DL|          11193|        12892|      -6|     -11|     11193|Cincinnati/Northe...|
|        19|        5|     DL|          10397|        15016|      -1|     -19|     10397|Hartsfield-Jackso...|
|

### Drop Duplicates()
        * It will drop duplicate records from existing DataFrame.

In [95]:
flights_df1 = flights_df.dropDuplicates()

In [96]:
flights_df1.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         6|        1|     WN|          10821|        10140|       1|     -22|
|         8|        1|     AA|          11298|        10140|       0|       6|
|        15|        1|     WN|          14747|        10140|      -6|       3|
|        27|        1|     AA|          11298|        10140|     113|     117|
|         7|        2|     OO|          12266|        10140|      -3|     -11|
|        28|        2|     WN|          14107|        10140|      -3|       0|
|        30|        2|     OO|          12266|        10140|      -4|     -11|
|         1|        3|     EV|          12266|        10140|     -11|     -26|
|         3|        3|     OO|          12892|        10140|      -4|      -2|
|        17|        3|     WN|          12892|      

#### Task1. Calculate percentage data lost after dropDuplicates()

In [97]:
(flights_df.count() - flights_df1.count()) * 100/ flights_df.count()

0.8249927006440348

### Describe()
        * Use describe() to show the summary statitics of numeric datatypes.
        * Summary includes count(), mean(), stddev(), min(), max().

In [102]:
flights_df1.describe().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|  count|           2696983|           2696983|2696983|           2696983|           2696983|           2674774|           2673185|
|   mean|15.798996508320593| 3.900369412784582|   null|12742.459424846207| 12742.85937657004|10.618575625454712|6.7272897311633875|
| stddev| 8.801267199135454|1.9864582421701973|   null|1502.0359941370625|1501.9939589817989|36.198308432512704| 38.75007476808384|
|    min|                 1|                 1|     9E|             10140|             10140|               -63|               -94|
|    max|                31|                 7|     YV|             15376|  

#### Summary()
        * Use the summary() method for detailed summary of columns.
        * Summary includes count(), mean(), stddev(), min(), 25%, 50%, 75%, max().

In [105]:
flights_df1.summary().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|  count|           2696983|           2696983|2696983|           2696983|           2696983|           2674774|           2673185|
|   mean|15.798996508320593| 3.900369412784582|   null|12742.459424846207| 12742.85937657004|10.618575625454712|6.7272897311633875|
| stddev| 8.801267199135454|1.9864582421701973|   null|1502.0359941370625|1501.9939589817989|36.198308432512704| 38.75007476808384|
|    min|                 1|                 1|     9E|             10140|             10140|               -63|               -94|
|    25%|                 8|                 2|   null|             11292|  

### Select all Categorial columns.

In [107]:
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import *

In [109]:
# Select categorical columns 
categorical_cols = [field.name for field in flights_df1.schema.fields if isinstance(field.dataType,StringType)]

In [110]:
categorical_cols

['Carrier']

#### Get frequency values for each categorical column

In [119]:
# In pyspark 'col' function is used to refer to a column in DataFrame.
# Where do you use it ? : In column operations
from pyspark.sql.functions import col

In [123]:
flights_df1.filter(col('ArrDelay') > 10).count()

664460

In [125]:
for column in categorical_cols:
    flights_df1.groupBy(column).count().show()

+-------+------+
|Carrier| count|
+-------+------+
|     UA|286010|
|     AA|288910|
|     EV|157218|
|     B6|121875|
|     DL|381601|
|     OO|159639|
|     F9| 35736|
|     YV| 52740|
|     US|232955|
|     MQ|112113|
|     HA| 17424|
|     AS| 68544|
|     FL| 92674|
|     VX| 34726|
|     WN|575090|
|     9E| 79728|
+-------+------+



### isnull() : used for checking the missing values.


In [129]:
# check for nulls in specific values
flights_df1.filter(col('ArrDelay').isNull()).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        17|        3|     DL|          14869|        14771|    null|    null|
|        11|        4|     EV|          12266|        13871|    null|    null|
|        19|        5|     EV|          11618|        11433|    null|    null|
|        10|        3|     EV|          13930|        13851|    null|    null|
|         9|        2|     EV|          11292|        14107|    null|    null|
|        18|        4|     AA|          13303|        12892|    null|    null|
|        11|        4|     AA|          13930|        11292|    null|    null|
|        18|        4|     AA|          11298|        14107|    null|    null|
|        16|        2|     AA|          11278|        13930|    null|    null|
|        17|        3|     AA|          13930|      

In [128]:
# check for number of nulls in specific column
flights_df1.filter(col('ArrDelay').isNull()).count()

23798

In [132]:
flights_df1.select(col('ArrDelay').isNull()).show()

+------------------+
|(ArrDelay IS NULL)|
+------------------+
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
+------------------+
only showing top 20 rows



In [142]:
flights_df1.select([count(when(isnull(column), column)).alias(column) for column in flights_df1.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   22209|   23798|
+----------+---------+-------+---------------+-------------+--------+--------+



In [209]:
### Create the same using alternative approach
for column in flights_df1.columns:
    print({column : flights_df1.filter(col(column).isNull()).count()})

{'DayofMonth': 0}
{'DayOfWeek': 0}
{'Carrier': 0}
{'OriginAirportID': 0}
{'DestAirportID': 0}
{'DepDelay': 22209}
{'ArrDelay': 23798}


In [211]:
flights_df1.schema.fields

[StructField(DayofMonth,IntegerType,true),
 StructField(DayOfWeek,IntegerType,true),
 StructField(Carrier,StringType,true),
 StructField(OriginAirportID,IntegerType,true),
 StructField(DestAirportID,IntegerType,true),
 StructField(DepDelay,IntegerType,true),
 StructField(ArrDelay,IntegerType,true)]

In [216]:
flights_df1.select([sum(col(column).isNull().cast('int')).alias(column) for column in flights_df1.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   22209|   23798|
+----------+---------+-------+---------------+-------------+--------+--------+



### fillna()
        * To replace or fill missing values of column with central tendancy(mean,median,mode).
        * Here we will replace missing values by 0s.

In [218]:
flights_df2 = flights_df1.fillna({'DepDelay':0, 'ArrDelay':0})

In [221]:
flights_df2.select([sum(col(column).isNull().cast('int')).alias(column) for column in flights_df2.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|       0|       0|
+----------+---------+-------+---------------+-------------+--------+--------+



### dropna()
        * Used to remove rows with null values

In [222]:
flights_df3 = flights_df1.dropna()

### Statistical Methods
        * mean(), median(), stddev(), quartiles.

In [226]:
flights_df1.select(round(mean(col('DepDelay'))).alias('Mean DepDelay')).show()

+-------------+
|Mean DepDelay|
+-------------+
|         11.0|
+-------------+



In [227]:
flights_df1.select(round(stddev(col('DepDelay'))).alias('STD DepDelay')).show()

+------------+
|STD DepDelay|
+------------+
|        36.0|
+------------+



In [230]:
flights_df1.select(round(variance(col('DepDelay'))).alias('Variance DepDelay')).show()

+---------------+
|Median DepDelay|
+---------------+
|         1310.0|
+---------------+



### To calculate median - Use approxQuantile(), expr()

In [239]:
flights_df1.approxQuantile('ArrDelay',[0.5],0.0001)[0]

-3.0

In [238]:
flights_df1.select(round(mean(col('DepDelay'))).alias('Mean DepDelay')).collect()[0][0]

11.0

### GroupBy() and agg()

In [241]:
flights_df1.groupBy('Carrier').agg(mean('DepDelay')).show()

+-------+------------------+
|Carrier|     avg(DepDelay)|
+-------+------------------+
|     UA|12.644186783024843|
|     AA|12.154096505870111|
|     EV| 14.52813602113455|
|     B6|12.675216069471794|
|     DL| 7.451940716867138|
|     OO| 7.954327121364983|
|     F9|12.142480802645592|
|     YV| 9.595018289496604|
|     US| 5.011879623272143|
|     MQ|15.612577198673192|
|     HA|1.5358414704192993|
|     AS|0.6606730403765751|
|     FL|10.227206113831024|
|     VX|14.416962353959326|
|     WN|12.930658050935614|
|     9E| 9.767838809034908|
+-------+------------------+



In [252]:
flights_df1.groupBy('Carrier').agg(mean('DepDelay').alias('Mean_DepDelay'),mean('ArrDelay').alias('Mean_ArrDelay')).show()

+-------+------------------+--------------------+
|Carrier|     Mean_DepDelay|       Mean_ArrDelay|
+-------+------------------+--------------------+
|     UA|12.644186783024843|   5.207155029152466|
|     AA|12.154096505870111|    7.22786703097812|
|     EV| 14.52813602113455|  10.501436641191532|
|     B6|12.675216069471794|   9.679335778153199|
|     DL| 7.451940716867138|  2.8085929091567747|
|     OO| 7.954327121364983|   6.447766785619039|
|     F9|12.142480802645592|  12.870312237233028|
|     YV| 9.595018289496604|   8.749505833107245|
|     US| 5.011879623272143|   3.957719324788726|
|     MQ|15.612577198673192|   14.27679662028746|
|     HA|1.5358414704192993|   1.534325271442523|
|     AS|0.6606730403765751|-0.27272328542814955|
|     FL|10.227206113831024|   7.277437501357486|
|     VX|14.416962353959326|   9.678802215555043|
|     WN|12.930658050935614|   8.368672739670938|
|     9E| 9.767838809034908|   4.931550031523011|
+-------+------------------+--------------------+


### withColumn
        * All column operations - used to create new column or modify an existing column in a DataFraame.
        * withColumn is also used for applying transformation or calculation.
        * df.withColumn(colName, col)

In [253]:
flights_df1.withColumn("Total Delay",col('DepDelay') + col('ArrDelay')).show()

+----------+---------+-------+---------------+-------------+--------+--------+-----------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Total Delay|
+----------+---------+-------+---------------+-------------+--------+--------+-----------+
|         6|        1|     WN|          10821|        10140|       1|     -22|        -21|
|         8|        1|     AA|          11298|        10140|       0|       6|          6|
|        15|        1|     WN|          14747|        10140|      -6|       3|         -3|
|        27|        1|     AA|          11298|        10140|     113|     117|        230|
|         7|        2|     OO|          12266|        10140|      -3|     -11|        -14|
|        28|        2|     WN|          14107|        10140|      -3|       0|         -3|
|        30|        2|     OO|          12266|        10140|      -4|     -11|        -15|
|         1|        3|     EV|          12266|        10140|     -11|     -26|        -37|

In [259]:
flights_df1.withColumn("IsDelay", when(col('DepDelay') >= 0, "Delay").otherwise("No Delay")).show()

+----------+---------+-------+---------------+-------------+--------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay| IsDelay|
+----------+---------+-------+---------------+-------------+--------+--------+--------+
|         6|        1|     WN|          10821|        10140|       1|     -22|   Delay|
|         8|        1|     AA|          11298|        10140|       0|       6|   Delay|
|        15|        1|     WN|          14747|        10140|      -6|       3|No Delay|
|        27|        1|     AA|          11298|        10140|     113|     117|   Delay|
|         7|        2|     OO|          12266|        10140|      -3|     -11|No Delay|
|        28|        2|     WN|          14107|        10140|      -3|       0|No Delay|
|        30|        2|     OO|          12266|        10140|      -4|     -11|No Delay|
|         1|        3|     EV|          12266|        10140|     -11|     -26|No Delay|
|         3|        3|     OO|  

In [260]:
### expr()
        

In [266]:
median_expr = expr(f"percentile_approx({'ArrDelay'},0.5)")
flights_df1.agg(median_expr.alias("Median")).collect()[0][0]

-3